In [1]:
#!/usr/bin/python
from os import listdir
from PIL import Image as PImage
import numpy as np
import random
import os
import glob
import cv2
import time

from __future__ import division

from keras import backend as K
K.set_image_dim_ordering('th')
from keras.callbacks import EarlyStopping, Callback
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import optimizers
from keras.models import Sequential, model_from_json
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Activation, Dropout, Flatten, Dense


### paths to training and testing data
#train_path = '/home/dens/Desktop/data'
#test_path = '/home/dens/Desktop/data'
train_path = os.path.join(os.getcwd(),'data')
test_path = os.path.join(os.getcwd(),'data')

### image dimensions
img_width, img_height = 40, 40
num_channels = 3
num_classes = 2


Using Theano backend.


In [3]:
#load image and resize it
def load_images(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_width, img_height), cv2.INTER_LINEAR)
    return resized

In [27]:
#load training folders (in this case I used 2 videos for training and 1 for testing)
def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Loading training images...')
    #name of the training folders
    folders = ["Fish2", "Fish1", "Nofish2", "Nofish1"] 
    #labes of the frames: 1- there is a fish, 0 - there is no fish
    labels = [1, 1, 0, 0]
    for ind, fld in enumerate(folders):
        print('Loading {} files (Index: {})'.format(fld, ind))
        path = os.path.join(train_path, fld, '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = load_images(fl)
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(labels[ind])

    print('Training data load time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id

In [28]:
def load_test():
    X_test = []
    X_test_id = []
    y_test = []
    
    folders = ["Fish3", "Nofish3"]
    labels = [1, 0]
    for ind, fld in enumerate(folders):
        print('Loading {} files (Index: {})'.format(fld, ind))
        path = os.path.join(test_path, fld, '*.jpg')
        files = sorted(glob.glob(path))
        for fl in files:
            flbase = os.path.basename(fl)
            img = load_images(fl)
            X_test.append(img)
            X_test_id.append(flbase)
            y_test.append(labels[ind])

    return X_test, y_test, X_test_id

In [29]:
#normalize data
train_data, train_target, train_id = load_train()
train_data = np.array(train_data, dtype=np.uint8)
train_target = np.array(train_target, dtype=np.uint8)

train_data = train_data.transpose((0, 3, 1, 2))
train_data = train_data.astype('float32')
train_data = train_data / 255

train_target = np_utils.to_categorical(train_target, 2)

Loading training images...
Loading Fish2 files (Index: 0)
Loading Fish1 files (Index: 1)
Loading Nofish2 files (Index: 2)
Loading Nofish1 files (Index: 3)
Training data load time: 4.8 seconds


In [30]:
test_data, test_target, test_id = load_test()
test_data = np.array(test_data, dtype=np.uint8)
test_target = np.array(test_target, dtype=np.uint8)

test_data = test_data.transpose((0, 3, 1, 2))

test_data = test_data.astype('float32')
test_data = test_data / 255

labels = test_target
test_target = np_utils.to_categorical(test_target, 2)

Loading Fish3 files (Index: 0)
Loading Nofish3 files (Index: 1)


In [31]:
# data augmentation, which performs horizontal and vertical flip as well as shifts

datagen = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    fill_mode='reflect')

datagen.fit(train_data)
#datagen.flow_from_directory(os.path.join(train_path, '1'), batch_size=32,save_to_dir=os.path.join(train_path, 'aug'),save_prefix='a',save_format='png', class_mode ='binary')

#i = 0
#for batch in datagen.flow_from_directory(train_path+'/1', target_size=(80,80),
#    class_mode='categorical', shuffle=False, batch_size=32,
#    save_to_dir=train_path+'/aug', save_prefix='hi'):
#
#    i += 1
#    if i > 20: # save 20 images
#        break  # otherwise the generator would loop indefinitely

In [32]:
# create model
model = Sequential()
nb_filters = 32
f_sz = 4
act = 'relu'

model.add(Convolution2D(nb_filters, f_sz, f_sz, activation=act, name='conv1_1', input_shape=(3, img_width, img_height)))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(nb_filters, f_sz, f_sz, activation=act, name='conv2_1'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(nb_filters, f_sz, f_sz, activation=act, name='conv3_1'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(32, activation=act))
#model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
#model.fit(train_data, train_target, nb_epoch=20, batch_size=32,  verbose=2)

# fits the model on batches with real-time data augmentation:
hist = model.fit_generator(datagen.flow(train_data, train_target, batch_size=32),
                    samples_per_epoch=len(train_data), nb_epoch=20)

# calculate predictions

Epoch 1/20
713/713 [==============================] - 2s - loss: 0.5548 - acc: 0.7700     

In [33]:
test_prediction = model.predict(test_data, batch_size=32, verbose=2)

In [34]:
predicted_labels = np.argmax(test_prediction, 1)
# print predicted_labels # the labels that we predicted
# print labels # original labels

# number of matches (right answers)
matches = (labels == predicted_labels).sum()
N = len(labels)
res = matches/N*100
print 'Correct prediction is for', res , '% of frames'

Correct prediction is for 85.9813084112 % of frames


In [35]:
A = []
A.append(hist.history['loss'])
A.append(hist.history['acc'])
A.append(res)
print A

[[0.55477093438112379, 0.54518877364810014, 0.54396419153975906, 0.54083475424564575, 0.53656260027624614, 0.5106644571914245, 0.50534678742728423, 0.48252432539453011, 0.51287682569712445, 0.4875192461642443, 0.48588720996450074, 0.48215670329825572, 0.46906215332835116, 0.49750237545157883, 0.4735513407251109, 0.47049586192755677, 0.48686706762745202, 0.48188438815120893, 0.48805551738759079, 0.47122491551314966], [0.76998597492175214, 0.76998597500534915, 0.76998597508894606, 0.76998597487995368, 0.7741935486378877, 0.79803646572174569, 0.80084151481010468, 0.80364656398206069, 0.77279102401011124, 0.80084151485190314, 0.79803646588893962, 0.80785413778179327, 0.80925666207518188, 0.80925666215877878, 0.80925666232597271, 0.81065918661936132, 0.80224403927068721, 0.80645161298682277, 0.80645161298682277, 0.81346423579131732], 85.981308411214954]


In [37]:
import csv
out = csv.writer(open("12.csv","w"), delimiter=',',quoting=csv.QUOTE_ALL)
out.writerow(A)